# Imports

In [1]:
import json
import os
import glob
from jsonschema import Draft7Validator

In [2]:
os.chdir('/media/sanketm/Data/Temp/CS314/csucs314fa20-repo/product/protocol') #path to the protocol repo to get the protocol json schemas
!git pull

Already up to date.


In [3]:
os.chdir('/media/sanketm/Data/Temp/CS314/csucs314fa20-repo/students/tests/sprint2') # path to the students repo
!git pull

Already up to date.


### Load Student Details from a json file

You will proabaly have to generate this file from the scripts present in the grading repository. Dave should give you the student team names. 

In [4]:
student_deets = json.load(open('/media/sanketm/Data/Temp/CS314/gradingandscripts/fullList.json','r'))
student_deets[0]

{'id': 831823987,
 'firstName': 'Alex',
 'lastName': 'Hooten',
 'teamNumber': 1,
 'email': 'ah00t13@rams.colostate.edu',
 'eID': 'ah00t13',
 'login': 'alex-hooten'}

In [5]:
student_eid = [a['eID'] for a in student_deets]
student_eid[:5]

['ah00t13', 'pdunton', 'nd0905', 'jgarc110', 'suyashh']

In [6]:
student_githubs = [a['login'] for a in student_deets]

In [7]:
student_githubs[:5]

['alex-hooten', 'prestondunton', 'nd0905', 'tacocabeza', 'suyashhiray']

### Get the tests submitted by the students

Above we've cd'd into a local git copy of the students repository and git pulled to get the updated repo. Now we collect all the student submitted tests jsons.

In [8]:
all_jsons = glob.glob('/media/sanketm/Data/Temp/CS314/csucs314fa20-repo/students/tests/sprint2/**.json') # get list of all json files

In [9]:
len(all_jsons)

191

In [10]:
all_jsons[2]

'/media/sanketm/Data/Temp/CS314/csucs314fa20-repo/students/tests/sprint2/ah00t13-distance.json'

In [11]:
with open(all_jsons[0],'r') as f:
    for line in f:
        print(line)

{

    "requestType": "distance",

    "requestVersion": 2,

    "place1": {

        "latitude": "39.9042",

        "longitude": "116.4074"

    },

    "place2": {

        "latitude": "39.7392",

        "longitude": "-104.9903"

    },

    "earthRadius": 6371,

    "distance": 10214

}



In [12]:
# with open(all_jsons[0],'r') as f:
#     contents = json.load(f)
# for key in contents:
#     print(key,":",contents[key])

We see that these files roughly are supposed to match the schema present in the product/protocol/RequestXXX.json. Where XXX is a type of request that the students need to program into their server(e.g. Trip, Distance, or Config. We will now pick up the json schemas for the distance request, which is what I'm testing here.

### Loading the json schemas for the requests
Here we load the json schemas for the requests that we want to test for the assignment. We get the schemas from the product/protocol repo.

In [13]:
with open('/media/sanketm/Data/Temp/CS314/csucs314fa20-repo/product/protocol/FindRequest.json','r') as f:
    find_jsonschema = json.load(f)
find_jsonschema

{'$id': 'https://example.com/address.schema.json',
 '$schema': 'http://json-schema.org/draft-07/schema#',
 'title': 'find',
 'description': 'find request/response',
 'type': 'object',
 'properties': {'requestVersion': {'description': 'the TIP protocol version',
   'type': 'integer',
   'minimum': 2},
  'requestType': {'description': 'the TIP object type should be locations',
   'type': 'string',
   'pattern': '^find$'},
  'match': {'description': 'an alphanumeric pattern used to find geographic locations.',
   'type': 'string',
   'pattern': '^[a-zA-Z0-9_ ]+$'},
  'limit': {'description': 'the maximum number of matching places that may be returned.',
   'type': 'integer',
   'minimum': 0},
  'found': {'description': 'the total number of matching places in the data source(s).',
   'type': 'integer',
   'minimum': 0},
  'places': {'description': 'list of places places found',
   'type': 'array',
   'items': {'type': 'object',
    'properties': {'name': {'type': 'string'},
     'latitude'

In [14]:
with open('/media/sanketm/Data/Temp/CS314/csucs314fa20-repo/product/protocol/DistanceRequest.json','r') as f:
    distance_jsonschema = json.load(f)
distance_jsonschema

{'$id': 'https://example.com/address.schema.json',
 '$schema': 'http://json-schema.org/draft-07/schema#',
 'title': 'distance',
 'description': 'distance request/response',
 'type': 'object',
 'properties': {'requestVersion': {'description': 'the TIP protocol version',
   'type': 'integer',
   'minimum': 2},
  'requestType': {'description': 'the TIP object type should be distance',
   'type': 'string',
   'pattern': '^distance$'},
  'place1': {'description': 'an object with the attributes to describe a place',
   'type': 'object',
   'properties': {'latitude': {'type': 'string',
     'pattern': '^[-+]?\\d+\\.?\\d*$'},
    'longitude': {'type': 'string', 'pattern': '^[-+]?\\d+\\.?\\d*$'}},
   'required': ['latitude', 'longitude'],
   'additionalProperties': True},
  'place2': {'description': 'an object with the attributes to describe a place',
   'type': 'object',
   'properties': {'latitude': {'type': 'string',
     'pattern': '^[-+]?\\d+\\.?\\d*$'},
    'longitude': {'type': 'string',

In [15]:
with open('/media/sanketm/Data/Temp/CS314/csucs314fa20-repo/product/protocol/TripRequest.json','r') as f:
    trip_jsonschema = json.load(f)
trip_jsonschema

{'$id': 'https://example.com/address.schema.json',
 '$schema': 'http://json-schema.org/draft-07/schema#',
 'title': 'trip',
 'description': 'trip request/response',
 'type': 'object',
 'properties': {'requestVersion': {'description': 'the TIP protocol version',
   'type': 'integer',
   'minimum': 3},
  'requestType': {'description': 'the TIP object type should be trip',
   'type': 'string',
   'pattern': '^trip$'},
  'options': {'description': 'options for this request',
   'type': 'object',
   'properties': {'title': {'type': 'string'},
    'earthRadius': {'type': 'string',
     'pattern': '(^\\+?[0-9]*[1-9][0-9]*(\\.[0-9]+)?$)|(^\\+?[0]+\\.[0-9]*[1-9][0-9]*$)'}},
   'required': ['earthRadius']},
  'places': {'description': 'list of places to visit',
   'type': 'array',
   'items': {'type': 'object',
    'properties': {'name': {'type': 'string'},
     'latitude': {'type': 'string',
      'pattern': '^[-+]?(?:90(?:(?:\\.0+)?)|(?:[0-9]|[1-8][0-9])(?:(?:\\.[0-9]+)?))$'},
     'longitude'

In [16]:
all_jsons[0]

'/media/sanketm/Data/Temp/CS314/csucs314fa20-repo/students/tests/sprint2/aericjw-distance.json'

### Testing the jsonschema package

In [17]:
all_jsons[0]

'/media/sanketm/Data/Temp/CS314/csucs314fa20-repo/students/tests/sprint2/aericjw-distance.json'

In [18]:
#install via pip install jsonschema, docs at https://python-jsonschema.readthedocs.io/en/stable
from jsonschema import Draft7Validator
for file in glob.glob('./*-distance.json'):
    with open(file,'r') as f:
        try:
            contents = json.load(f)
            try:
                Draft7Validator(distance_jsonschema).validate(contents)
            except:
                print(f"file {file} does not match the schema")
        except:
            print(f"File {file} is not valid json")

file ./cmar1299-distance.json does not match the schema
file ./fosterm4-distance.json does not match the schema
file ./gdeyoung-distance.json does not match the schema
File ./hlingle-distance.json is not valid json
File ./jabayless-distance.json is not valid json
File ./mbauers-distance.json is not valid json
file ./mikep99-distance.json does not match the schema
file ./moise-distance.json does not match the schema
file ./rylied-distance.json does not match the schema
File ./smunoz-distance.json is not valid json
file ./thomas41-distance.json does not match the schema
file ./wckelly-distance.json does not match the schema


In [19]:
#install via pip install jsonschema, docs at https://python-jsonschema.readthedocs.io/en/stable
from jsonschema import Draft7Validator
for file in glob.glob('./*-find.json'):
    with open(file,'r') as f:
        try:
            contents = json.load(f)
            try:
                Draft7Validator(find_jsonschema).validate(contents)
            except:
                print(f"file {file} does not match the schema")
        except:
            print(f"File {file} is not valid json")

file ./barelads-find.json does not match the schema
File ./fonsecao-find.json is not valid json
file ./idmunro19-find.json does not match the schema
File ./jabayless-find.json is not valid json
File ./kwreindl-find.json is not valid json
File ./nguyenh-find.json is not valid json
file ./rbclark-find.json does not match the schema
File ./xinyi99-find.json is not valid json


In [20]:
# #install via pip install jsonschema, docs at https://python-jsonschema.readthedocs.io/en/stable
# from jsonschema import Draft7Validator
# for file in glob.glob('./*.json'):
#     with open(file,'r') as f:
#         try:
#             contents = json.load(f)
#             try:
#                 Draft7Validator(trip_jsonschema).validate(contents)
#                 print(f"file {file} matches the schema")
#             except:
#                 print(f"file {file} does not match the schema")
#         except:
#             print(f"File {file} is not valid json")

### Writing the whole loop

TODO: Make robust by removing hardcoded request type checks. instead read all requests.jsons in product/protocol and check if the submitted file contents request type tag matches any names then match those names schemas.

In [21]:
# unhandled = [x for x in all_jsons if len(x.split('/')[-1].split("-")) > 2]
# for x in unhandled: print(x)
# print(len(all_jsons))
# all_jsons = [a for a in all_jsons if a not in unhandled]
# print(len(all_jsons))

In [46]:
os.chdir('/media/sanketm/Data/Temp/CS314/csucs314fa20-repo/students/tests/sprint2/')
all_jsons = glob.glob('./**.json') # get list of all json files
all_jsons[0]

'./aericjw-distance.json'

In [47]:
student_dict = {}

for file in all_jsons:    
    with open(file, 'r') as f:
        #eids are always one word so this works
        eid = file.split("/")[-1].split("-")[0] # splitting. / shlok - distance.json to get shlok
        req = file.split("/")[-1].split("-")[1].split(".")[0]  # splitting. / shlok - distance.json to get distance
        eid = eid.lower()
        req = req.lower()
        
        student_rec = student_dict.get(eid, {})
        
        if eid in student_eid:

            ## valid file_name , give points for valid filename,
            ##Then check valid json
            ##otherwise skip the valid filename points and jump to json load and schema validations conditions
            if req == 'find':
                print(f"{eid} has find")
                student_rec['submitted find with eid'] = 1
                student_rec['find_filename'] = file
                
            if req == 'distance':
                print(f"{eid} has distance")
                student_rec['submitted distance with eid'] = 1
                student_rec['distance_filename'] = file
            
            if req == 'trip':
                print(f"{eid} has trip")
                student_rec['submitted trip with eid'] = 1
                student_rec['trip_filename'] = file
        
        elif eid in student_githubs: 
            #Github ids can be two words like 'evan-fitzpatrick' so we have to handle the file name differently.
            if req == 'find':
                print(f"{eid} has find")
                student_rec['submitted find with github id'] = 1
                student_rec['find_filename'] = file
                
            if req == 'distance':
                print(f"{eid} has distance")
                student_rec['submitted distance with github id'] = 1
                student_rec['distance_filename'] = file
            
            if req == 'trip':
                print(f"{eid} has trip")
                student_rec['submitted trip with github id'] = 1
                student_rec['trip_filename'] = file
        else:
            # student has submitted file with other file name like github_request or fullname_request
            student_rec['invalid_file'] = 1
            student_rec['comment'] = "Student's filename: "+ file +"was not in the format eid_request or github_request, so was not valid." 
            if student_rec.get('invalid_filename',{}) == {}: 
                student_rec['invalid_filename'] = str(file)
            else:
                student_rec['invalid_filename1'] = str(file)

        try:
#             print(f"Loading JSON {f}")
            json_file = json.load(f)
            if json_file:
                
                if json_file['requestType'] == 'find':
                    student_rec['valid find json'] = 1
                    try:
                        Draft7Validator(find_jsonschema).validate(json_file)
                        student_rec['find match schema'] = 1
                    except:
                        print(f"file {file} does not match the schema")
                        student_rec['find match schema'] = 0
                    
                if json_file['requestType'] == 'distance':
                    student_rec['valid distance json'] = 1
                    try:
                        Draft7Validator(distance_jsonschema).validate(json_file)
                        student_rec['distance match schema'] = 1
                    except:
                        print(f"file {file} does not match the schema")
                        student_rec['distance match schema'] = 0

                if json_file['requestType'] == 'trip':
                    student_rec['valid trip json'] = 1
                    try:
                        Draft7Validator(trip_jsonschema).validate(json_file)
                        student_rec['trip match schema'] = 1
                    except:
                        print(f"file {file} does not match the schema")
                        student_rec['trip match schema'] = 0

        except:
            print(f"{file} is invalid ")
            if req == 'find':
                student_rec['invalid find json'] = 1

            if req == 'distance':
                student_rec['invalid distance json'] = 1

            if req == 'trip':
                student_rec['invalid trip json'] = 1

            student_rec['comment'] = "json file could not load so not valid." 
            student_rec['invalid_file'] = 1
            if student_rec.get('invalid_filename',{}) == {}: 
                student_rec['invalid_filename'] = file
            else:
                student_rec['invalid_filename1'] = str(file)

        student_dict[eid] = student_rec

aericjw has distance
aericjw has find
ah00t13 has distance
ah00t13 has find
alecmilt has distance
alecmilt has find
augustai has distance
augustai has find
backlira has distance
backlira has find
barelads has distance
barelads has find
file ./barelads-find.json does not match the schema
bcombs18 has distance
bcombs18 has find
benschra has distance
benschra has find
bobbycsu has find
braedens has distance
braedens has find
camconn has distance
camconn has find
caustin has distance
caustin has find
cessna17 has distance
cessna17 has find
ch85 has distance
ch85 has find
chenhui has distance
chenhui has find
cmar1299 has distance
file ./cmar1299-distance.json does not match the schema
cmar1299 has find
colecole has distance
connorsw has distance
connorsw has find
criech5 has distance
criech5 has find
cwernsma has distance
cwernsma has find
darinh has distance
darinh has find
dbilbao has distance
dbilbao has find
deionus has distance
deionus has find
drew21 has distance
drew21 has find
eber

### Handling names that are not in the eids list or the github ids list

In [48]:
# for name in student_eid:
#     if name not in student_dict.keys():
#         student_rec = {}
#         student_rec['submitted find'] = 0
#         student_rec['submitted distance'] = 0
#         student_dict[name] = student_rec

In [49]:
# print(student_dict)

### Cleanup and writing to file

In [50]:
len(student_dict)

97

In [51]:
for key in ['cs314','kaseda','sanketm','zhouxy','dzuigas','dziugas']:
    if student_dict.get(key,[]) != []:
        del(student_dict[key])

In [52]:
len(student_dict)

97

In [53]:
import pandas as pd
df = pd.DataFrame().from_dict(student_dict)
df = df.fillna(0)
df = df.transpose()

In [54]:
df

,submitted distance with github id,distance_filename,valid distance json,distance match schema,submitted find with github id,find_filename,valid find json,find match schema,submitted distance with eid,submitted find with eid,invalid_file,comment,invalid_filename,invalid find json,invalid_filename1,invalid distance json
aericjw,1,./aericjw-distance.json,1,1,1,./aericjw-find.json,1,1,0,0,0,0,0,0,0,0
ah00t13,0,./ah00t13-distance.json,1,1,0,./ah00t13-find.json,1,1,1,1,0,0,0,0,0,0
alecmilt,0,./alecmilt-distance.json,1,1,0,./alecmilt-find.json,1,1,1,1,0,0,0,0,0,0
augustai,0,./augustai-distance.json,1,1,0,./augustai-find.json,1,1,1,1,0,0,0,0,0,0
backlira,0,./backlira-distance.json,1,1,0,./backlira-find.json,1,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wckelly,0,./wckelly-distance.json,1,0,0,./wckelly-find.json,1,1,1,1,0,0,0,0,0,0
wyatt,0,0,1,1,0,0,1,1,0,0,1,Student's filename: ./wyatt-temple-find.jsonwa...,./wyatt-temple-distance.json,0,./wyatt-temple-find.json,0
xinyi99,0,./xinyi99-distance.json,1,1,0,./xinyi99-find.json,0,0,1,1,1,json file could not load so not valid.,./xinyi99-find.json,1,0,0
yiboxu,0,./yiboxu-distance.json,1,1,0,./yiboxu-find.json,1,1,1,1,0,0,0,0,0,0


In [55]:
os.chdir('/media/sanketm/Data/Temp/CS314/gradingandscripts')
df.to_csv('eid_sprint2_tests.csv')

In [32]:
# with open('eid_sprint2_tests.csv', 'w') as f:
#     for key in student_dict.keys():
#         f.write("%s,%s\n"%(key,student_dict[key]))